In [ ]:
%pip install sqlalchemy psycopg2
%pip install python-dotenv

In [1]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
from psycopg2 import OperationalError

# Load environment variables from the .env file
load_dotenv(dotenv_path="./variable.env")

# Retrieve database connection details from environment variables
db_user = os.getenv("db_user")
db_password = os.getenv('db_password')
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
db_name = os.getenv("db_name")

# Establish a connection to the PostgreSQL database
connection = psycopg2.connect(user=db_user,
                              password=db_password,
                              host=db_host,
                              port=db_port,
                              database=db_name)

# Test connection
connection

# Create a cursor to execute SQL queries
cursor = connection.cursor()

# Retrieve the list of hotel IDs from the environment variable
hotel_ids = os.getenv('HOTEL_IDS')

# Check if the environment variable contains hotel IDs
if hotel_ids:
    hotel_ids = hotel_ids.split(',')
else:
    raise ValueError("La variable d'environnement 'HOTEL_IDS' est manquante ou vide.")


# Iterate over each hotel ID to execute queries and export data
for hotel_id in hotel_ids:

    # SQL query to retrieve FAQ data for the hotel
    query_faq = f"""
    SELECT hotels.id, hotels."name", faq_categories_translation."name" as "category",
           faq_questions_translation.question, hotel_faq_questions.response
    FROM hotel_faq_questions
    INNER JOIN faq_questions_translation
        ON faq_questions_translation.faq_question_id = hotel_faq_questions.id_question
        AND faq_questions_translation.language = 'en'
    INNER JOIN faq_categories_translation
        ON faq_categories_translation.faq_category_id = hotel_faq_questions.faq_category_id
        AND faq_categories_translation.language = 'en'
    INNER JOIN hotels
        ON hotels.id = hotel_faq_questions.hotel_id
    WHERE hotel_id = '{hotel_id}';
    """

    # SQL query to retrieve services data for the hotel
    query_services = f"""
    SELECT hotels.id, hotels.name, service_categories_translation."name" as "service category",
           services."name" as "service name", hotel_services_translation."type",
           hotel_services_translation."description", hotel_services.opening_hours
    FROM hotel_services
    INNER JOIN service_categories_translation
        ON service_categories_translation.service_category_id = hotel_services.service_category_id
        AND service_categories_translation."language" = 'en'
    INNER JOIN services
        ON services.id = hotel_services.service_id
    INNER JOIN hotel_services_translation
        ON hotel_services_translation.hotel_service_id = hotel_services.id
        AND hotel_services_translation."language" = 'en'
    INNER JOIN hotels
        ON hotels.id = hotel_services.hotel_id
    WHERE hotel_id = '{hotel_id}';
    """

    # Execute the FAQ query and load results into a pandas DataFrame
    df_faq = pd.read_sql_query(query_faq, connection)

    # Execute the Services query and load results into a pandas DataFrame
    df_services = pd.read_sql_query(query_services, connection)

    # Create a subdirectory to store CSV files for the hotel
    hotel_directory = f"Data/{hotel_id}"
    os.makedirs(hotel_directory, exist_ok=True)

    # Define file paths for the CSV files
    chemin_csv_faq = os.path.join(hotel_directory, "faq_data.csv")
    chemin_csv_services = os.path.join(hotel_directory, "services_data.csv")

    # Export the DataFrames to CSV files
    df_faq.to_csv(chemin_csv_faq, index=False)
    df_services.to_csv(chemin_csv_services, index=False)

  # Print confirmation messages
    print(f"FAQ data for hotel {hotel_id} has been exported to {chemin_csv_faq}")
    print(f"Services data for hotel {hotel_id} has been exported to {chemin_csv_services}")

FAQ data for hotel 00bc4c1e-69d8-4dc9-a317-4847ab13a2ca has been exported to Data/00bc4c1e-69d8-4dc9-a317-4847ab13a2ca\faq_data.csv
Services data for hotel 00bc4c1e-69d8-4dc9-a317-4847ab13a2ca has been exported to Data/00bc4c1e-69d8-4dc9-a317-4847ab13a2ca\services_data.csv
FAQ data for hotel 0126e2f8-5094-4940-a4c3-72e64152c6e9 has been exported to Data/0126e2f8-5094-4940-a4c3-72e64152c6e9\faq_data.csv
Services data for hotel 0126e2f8-5094-4940-a4c3-72e64152c6e9 has been exported to Data/0126e2f8-5094-4940-a4c3-72e64152c6e9\services_data.csv
FAQ data for hotel 189f4acc-f035-4ca2-8d26-5cdfd8560949 has been exported to Data/189f4acc-f035-4ca2-8d26-5cdfd8560949\faq_data.csv
Services data for hotel 189f4acc-f035-4ca2-8d26-5cdfd8560949 has been exported to Data/189f4acc-f035-4ca2-8d26-5cdfd8560949\services_data.csv
FAQ data for hotel 55ae1399-d076-40b3-bc7f-edabf16100d4 has been exported to Data/55ae1399-d076-40b3-bc7f-edabf16100d4\faq_data.csv
Services data for hotel 55ae1399-d076-40b3-bc7

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_300\2370829074.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_faq = pd.read_sql_query(query_faq, connection)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_300\2370829074.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_services = pd.read_sql_query(query_services, connection)
